In [1]:
import pandas as pd
from datasets import load_dataset_builder, get_dataset_config_names, load_dataset
from huggingface_hub import HfApi
import numpy as np
import pdb
import logging
import re

# 0. Hugging Face login
Necessary only got 'Gated' datasets on hugging face

In [2]:
from huggingface_hub import login
login()

## 1. Data Visualization

In [3]:
#TODO
mt_hf_df = pd.read_csv('data/language_pairs_hf.csv')

In [4]:
mt_hf_df

,Author/Dataset,Language Pair,# Train Set,# Development Set,# Test Set
0,larryvrh/CCMatrix-v1-Ja_Zh-filtered,zh-ja,5686275,0,0
1,achrafothman/aslg_pc12,ase-en,87710,0,0
2,Helsinki-NLP/bible_para,de-en,62195,0,0
3,Helsinki-NLP/bible_para,en-fr,62195,0,0
4,Helsinki-NLP/bible_para,en-es,62191,0,0
...,...,...,...,...,...
4052,ziozzang/deepl-trans-FR-KO,fr-ko,10281,0,0
4053,ziozzang/deepl-trans-ID-KO,id-ko,5354,0,0
4054,ziozzang/deepl-trans-IT-KO,it-ko,15817,0,0
4055,ziozzang/deepl-trans-PT-KO,pt-ko,5043,0,0


## 2. Update ```language_pairs_external.csv```

In [5]:
from utils import update_pairs
from math import factorial

In [6]:
mtex_pair = pd.read_csv('data/language_pairs_external.csv')
mtex_pair.head()

,Author/Dataset,Language Pair,# Train set,# Development set,# Test set
0,masakhane/mafand,en-am,0,899,1037
1,masakhane/mafand,en-ha,5865,1300,1500
2,masakhane/mafand,en-ig,6998,1500,1500
3,masakhane/mafand,en-rw,0,460,1006
4,masakhane/mafand,en-lg,4075,1500,1500


In [7]:
# Take a look at the helper function for ```update pairs```
help(update_pairs)

Help on function update_pairs in module utils:

update_pairs(dataset_author, supp_langs, n_rows, dtype='Multiway', save_df=False)
    Function to update the available language pairs in multilingual datasets.
    
        :param dataset_author: ID from Hugging Face/external dataset
        :params supp_langs: Languages in dataset
        :params n_rows: Number of rows in df in the form (train, val, test); 
                    - List object (parallel, multilingual)
                    - Dictionary object (English-Centric)
        :params dtype: The type of multilingual dataset (Multiway, English-Centric, Pivot-Based)
        :params save_df: Save modified CSV file 
        :return: dataframe



### Multiway example using  [HornMT](https://github.com/asmelashteka/HornMT) dataset from GitHub
The number of language pairs for a multiway is obtained with the permutation formula.
- Change save to True to save your changes

In [8]:
def permutations(n, r):
    '''Returns the number of permutations.'''
    return int(factorial(n) / factorial(n-r))

val = permutations(6, 2)
print(f"There will be {val} distinct pairs for the HornMT dataset.")

There will be 30 distinct pairs for the HornMT dataset.


In [9]:
# Arguments for function
data_auth = 'LesanAI/HornMT' # if external check the main contributor to the dataset
langs = ['aa', 'am', 'en', 'om', 'so', 'ti']
rows = [0, 0, 2030] # multiway datasets will have the same n_rows
d_type = 'Multiway'
save = False # change to True; param is False only for demonstration purposes

In [10]:
df = update_pairs(data_auth, langs, rows, d_type, save)
df.tail(30) # 30 distinct pairs

,Author/Dataset,Language Pair,# Train set,# Development set,# Test set
220,LesanAI/HornMT,am-aa,0,0,2030
221,LesanAI/HornMT,en-aa,0,0,2030
222,LesanAI/HornMT,om-aa,0,0,2030
223,LesanAI/HornMT,so-aa,0,0,2030
224,LesanAI/HornMT,ti-aa,0,0,2030
225,LesanAI/HornMT,aa-am,0,0,2030
226,LesanAI/HornMT,en-am,0,0,2030
227,LesanAI/HornMT,om-am,0,0,2030
228,LesanAI/HornMT,so-am,0,0,2030
229,LesanAI/HornMT,ti-am,0,0,2030


### English-Centric example using  [Samantar](https://huggingface.co/datasets/ai4bharat/samanantar) dataset from Hugging Face
There will be *n-1* number of language pairs for an English-Centric dataset. There will be 11 unique pairs for Samantar.
- If the dataset doesn't exist in the ```mt_hf.csv``` dataset then you will manually add the dataset to ```mt_hf_external.csv```

In [11]:
# Arguments for function
data_auth = 'ai4bharat/samanantar' # if external check the main contributor to the dataset
configs = get_dataset_config_names(data_auth)
print(configs)

['as', 'bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'or', 'pa', 'ta', 'te']


In [12]:
langs = configs.copy()
langs.append('en') # ensure English is in the list
d_type = 'English-Centric'
save = False # change to True; param is False only for demonstration purposes
print(langs)

['as', 'bn', 'gu', 'hi', 'kn', 'ml', 'mr', 'or', 'pa', 'ta', 'te', 'en']


English-Centric datasets may not have the same n_rows! Therefore we'll create a dictionary for each unique language pair containing their (train, validation, test) pair.

In [13]:
# this is easy if the config is similar to Samantar 
pairs = {}
for config in configs:
    rows = [0, 0, 0]
    builder = load_dataset_builder(data_auth, config)
    info = builder.info
    for split in info.splits:
        if split.startswith('train'):
            rows[0] = info.splits[split].num_examples
        if split.startswith('val'):
            rows[1] = info.splits[split].num_examples
        if split.startswith('test'):
            rows[2] = info.splits[split].num_examples
            
    pairs[config] = rows

In [14]:
# Otherwise you'll have to manually enter the number of rows or think of a programmatic solution.
test = {}
test['as'] = [141226, 0, 0]
test['bn'] = [8604579, 0, 0]

In [15]:
print("There will be 11 distinct pairs for the Samanantar dataset.")

There will be 11 distinct pairs for the Samanantar dataset.


In [16]:
df = update_pairs(data_auth, langs, pairs, d_type, save)
df.tail(11) 

,Author/Dataset,Language Pair,# Train set,# Development set,# Test set
220,ai4bharat/samanantar,en-as,141227,0,0
221,ai4bharat/samanantar,en-bn,8604580,0,0
222,ai4bharat/samanantar,en-gu,3067790,0,0
223,ai4bharat/samanantar,en-hi,10125706,0,0
224,ai4bharat/samanantar,en-kn,4093524,0,0
225,ai4bharat/samanantar,en-ml,5924426,0,0
226,ai4bharat/samanantar,en-mr,3627480,0,0
227,ai4bharat/samanantar,en-or,998228,0,0
228,ai4bharat/samanantar,en-pa,2980383,0,0
229,ai4bharat/samanantar,en-ta,5264867,0,0


### Simple parallel example using  [Filtered-Japanese-English-Parallel-Corpus](https://github.com/asmelashteka/HornMT) dataset from Hugging Face
A simple parallel dataset contains only 2 language pairs.

In [42]:
data_auth = 'Moleys/Filtered-Japanese-English-Parallel-Corpus' # if external check the main contributor to the dataset
langs = ['ja', 'en']
rows = [10739509, 0, 0] # multiway datasets will have the same n_rows
d_type = 'Simple Parallel'
save = False # change to True; param is False only for demonstration purposes

In [43]:
df = update_pairs(data_auth, langs, rows, d_type, save)
df.tail(1)

,Author/Dataset,Language Pair,# Train set,# Development set,# Test set
220,Moleys/Filtered-Japanese-English-Parallel-Corpus,ja-en,10739509,0,0


## 3. Misc

In [17]:
# re.fullmatch(pattern, configs[12])

In [43]:
import re

# Define the pattern for basic ISO language code pairs
# pattern = r'^[a-z]{2,3}(-|2)[a-z]{2,3}$'
pattern = r'[a-z]{2,3}((_|-)\w+)?(-|2)[a-z]{2,3}((_|-)\w+)?' # new pattern!

# Example language code pairs
codes = ['en-es', 'fr-de', 'zh-en', 'EN-es', 'eng-es_AM', 'ara_blahblah', 'iwslt14_de_en', 'amh_Ethi-arb_Arab']

# Filter valid codes
valid_codes = [code for code in codes if re.fullmatch(pattern, code)]

print(valid_codes)  # Output: ['en-es', 'fr-de', 'zh-en']


['en-es', 'fr-de', 'zh-en']


In [179]:
pattern = r'[a-z]{2,3}-[a-z]{2,3}(_-)?.*'
# pattern = 'en-zh'

In [182]:
string = "aya_dataset"
re.search(pattern, string)
# help(re.match)